In [1]:
import os
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data as data
import torchvision
from torchvision import datasets, transforms

In [2]:
import sys

COLAB = 'google.colab' in str(get_ipython())
TRANSFORMS = True
HAS_GPU = torch.cuda.is_available()

if COLAB: # running on Colab
    
    from google.colab import drive
    drive.mount('/content/drive')
    gdrive_path = '/content/drive//MyDrive/DL4CV-2022/project-I/'
    sys.path.append(gdrive_path)
    
    if HAS_GPU:
        print("Using GPU")
else:
    path = 'data'
    HAS_GPU = False
    print("Using CPU")
    
import utils.helpers as utils
import loader.gtsrb_data as dataset

# If we have cuda running then accomodate these datatype.
FloatTensor = torch.cuda.FloatTensor if HAS_GPU else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if HAS_GPU else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if HAS_GPU else torch.ByteTensor
Tensor = FloatTensor

Using CPU


In [3]:
parser = argparse.ArgumentParser()

# Load and train arguments
parser.add_argument('--data_dir_train', type=str, default='data/Final_Training/Images',
                   help='data directory containing training image class folders')
parser.add_argument('--data_dir_test', type=str, default='data/Final_Test',
                   help='data directory containing test images')
parser.add_argument('--annotations_file', type=str, default='data/Final_Training/Annotations',
                   help='data directory containing class annotations file')
parser.add_argument('--data_dir_val', type=str, default='data/Final_Validation',
                   help='data directory containing validation images')
parser.add_argument('--log_dir', type=str, default='logs',
                   help='directory containing logs')
parser.add_argument('--save_dir', type=str, default='save',
                   help='directory to store checkpointed models')
parser.add_argument('--batch_size', type=int, default=50,
                   help='input batch size for training')
parser.add_argument('--num_epochs', type=int, default=5,
                   help='number of epochs to train')
parser.add_argument('--learning_rate', type=float, default=0.0001,
                    help='learning rate (default: 0.0001)')
parser.add_argument('--seed', type=int, default=1,
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10,
                    help='how many batches to wait before logging training status')
parser.add_argument('--save_every', type=int, default=1000,
                   help='save frequency')
parser.add_argument('--decay_rate', type=float, default=0.97,
                   help='decay rate for rmsprop')
parser.add_argument('--gpu_mem', type=float, default=0.666,
                   help='%% of gpu memory to be allocated to this process. Default is 66.6%%')
parser.add_argument('--init_from', type=str, default=None,
                   help="""continue training from saved model at this path. Path must contain files saved by previous training process:
                        'config.pkl'        : configuration;
                        'words_vocab.pkl'   : vocabulary definitions;
                        'checkpoint'        : paths to model file(s) (created by tf).
                                              Note: this file contains absolute paths, be careful when moving files around;
                        'model.ckpt-*'      : file(s) with model definition (created by tf)
                    """)
args = parser.parse_args(args=[])
print(args)

Namespace(annotations_file='data/Final_Training/Annotations', batch_size=50, data_dir_test='data/Final_Test', data_dir_train='data/Final_Training/Images', data_dir_val='data/Final_Validation', decay_rate=0.97, gpu_mem=0.666, init_from=None, learning_rate=0.0001, log_dir='logs', log_interval=10, num_epochs=5, save_dir='save', save_every=1000, seed=1)


In [4]:
#transform=TRANSFORM_IMG

if COLAB:
    TRAIN_DATA_PATH = gdrive_path+args.data_dir_train
    VAL_DATA_PATH = gdrive_path+args.data_dir_val
    TEST_DATA_PATH = gdrive_path+args.data_dir_test
    print(gdrive_path+args.data_dir_test)
    print(gdrive_path+args.annotations_file)

else:
    TRAIN_DATA_PATH = os.getcwd()+'/'+args.data_dir_train
    VAL_DATA_PATH = os.getcwd()+'/'+args.data_dir_val
    TEST_DATA_PATH = os.getcwd()+'/'+args.data_dir_test
    
BATCH_SIZE = args.batch_size
EPOCHS = args.num_epochs
BATCH_SIZE = args.batch_size
LEARNING_RATE = args.learning_rate

In [5]:
if TRANSFORMS:
    from utils.loader import transforms


    # Apply data transformation to augment training imageset.
    train_loader = torch.utils.data.DataLoader(datasets.ImageFolder(TRAIN_DATA_PATH,
                                                                     transform=transforms.data_transforms), 
                                               batch_size=args.batch_size, 
                                               shuffle=True, num_workers=2, 
                                               pin_memory=HAS_GPU)
    
    val_loader = torch.utils.data.DataLoader(datasets.ImageFolder(VAL_DATA_PATH,
                                                                  transform=transforms.data_transforms),
                                             batch_size=args.batch_size, 
                                             shuffle=False, num_workers=2, 
                                             pin_memory=HAS_GPU)
    
    # Apply data transformations to augment the training imageset.
   # train_loader = torch.utils.data.DataLoader(
   #     torch.utils.data.ConcatDataset([torchvision.datasets.ImageFolder(TRAIN_DATA_PATH,
   #     transform=transforms.data_transforms),torchvision.datasets.ImageFolder(TRAIN_DATA_PATH,
   #     transform=transforms.data_jitter_contrast),torchvision.datasets.ImageFolder(TRAIN_DATA_PATH,
   #     transform=transforms.data_grayscale),torchvision.datasets.ImageFolder(TRAIN_DATA_PATH,
   #     transform=transforms.data_translate)]), 
   #         batch_size=args.batch_size, 
   #         shuffle=True, num_workers=2, 
   #         pin_memory=HAS_GPU)

else: # In this case train=39209, test=12630, val=3870
    train_data        = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=None)
    train_data_loader = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2)
    test_data         = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=None)
    test_data_loader  = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    val_data          = torchvision.datasets.ImageFolder(root=VAL_DATA_PATH, transform=None)
    val_data_loader   = data.DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2)
    
# Order: data_transforms, data_jitter_contrast, data_grayscale, data_translate  


In [6]:
print(len(train_data))
print(len(test_data))
print(len(val_data))

39209
12630
3870


In [7]:
train_class_dict = train_data.class_to_idx
val_class_dict = val_data.class_to_idx
test_class_dict = test_data.class_to_idx # A 'single' class

In [14]:
# Neural Network and Optimizer
from model.models import Net as NN
model = NN()

if HAS_GPU:
    model.cuda()
    
optimizer = optim.Adam(filter(lambda p: p.requires_grad,
                              model.parameters()),
                              lr=args.learning_rate)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 'min',
                                                 patience=5,
                                                 factor=0.5,
                                                 verbose=True)

In [15]:
def train(epoch):
    model.train()
    correct = 0
    training_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        if use_gpu:
            data = data.cuda()
            target = target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        max_index = output.max(dim = 1)[1]
        correct += (max_index == target).sum()
        training_loss += loss
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss per example: {:.6f}\tLoss: {:.6f}'.format(
                        epoch, batch_idx * len(data), len(train_loader.dataset),
                        100. * batch_idx / len(train_loader), loss.data.item()/(args.batch_size * args.log_interval),
                        loss.data.item()))
            print('\nTraining set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
                training_loss / len(train_loader.dataset), correct, len(train_loader.dataset),
                100. * correct / len(train_loader.dataset)))
    